In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.impute import KNNImputer

#### Data cleaning techniques



In [22]:

def clean_data(df):
    #Remove duplicates
    df=df.drop_duplicates()

    #handle outliers
    df['age']=df['age'].clip(0,120)

    #Correct structural errors
    df['date']=pd.to_datetime(df['date'],errors='coerce')

    # Standardize text data (e.g., for ’category’ column)
    df['category']=df['category'].str.lower().str.strip()

    #Type conversion
    df['price']=pd.to_numeric(df['price'],errors='coerce')

    return df

#Example Usage


data = pd.DataFrame({
    'age': [25, 30, 5, 150, 40],
    'date': ['2023-01-01', '01/15/2023', 'invalid', '2023-02-28', '03-15-2023'],
    'category': ['Electronics', 'electronics', 'Clothing', 'FOOD', 'food'],
    'price': ['100', '208.5', 'N/A', '300', '150.75']
})

print(data)
print('----')
cleaned_data=clean_data(data)
print(cleaned_data)


   age        date     category   price
0   25  2023-01-01  Electronics     100
1   30  01/15/2023  electronics   208.5
2    5     invalid     Clothing     N/A
3  150  2023-02-28         FOOD     300
4   40  03-15-2023         food  150.75
----
   age       date     category   price
0   25 2023-01-01  electronics  100.00
1   30        NaT  electronics  208.50
2    5        NaT     clothing     NaN
3  120 2023-02-28         food  300.00
4   40        NaT         food  150.75


#### Handling Missing Data

In [23]:
def handle_missing_data(df):
    # Identify columns with missing data
    columns_with_missing = df.columns[df.isnull().any()].tolist()

    for column in columns_with_missing:
        missing_percentage = df[column].isnull().sum() / len(df) * 100

        if missing_percentage < 5:
            # For Low percentage of missing data, use mean imputation
            df[column].fillna(df[column].mean(), inplace=True)

        elif missing_percentage < 15:
            # For moderate missing data, use median imputation
            df[column].fillna(df[column].median(), inplace=True)

        else:
            # For high percentage of missing data, use KNN imputation
            # Round off to 1 integers for more consistency
            imputer = KNNImputer(n_neighbors=5)
            df[column] = np.round(imputer.fit_transform(df[[column]]),1)

    return df

# Example usage
data = pd.DataFrame({
    'A': [1, 2, np.nan, 4, 5],
    'B': [np.nan, 2, 3, np.nan, 5],
    'C': [1, 2, 3, 4, np.nan]
})

cleaned_data = handle_missing_data(data)
print(cleaned_data)


     A    B    C
0  1.0  3.3  1.0
1  2.0  2.0  2.0
2  3.0  3.0  3.0
3  4.0  3.3  4.0
4  5.0  5.0  2.5


#### Data normalization and standardization

In [24]:
def normalize_and_standardize(df):
    # Create copies of the dataframe for different scaling methods
    df_minmax = df.copy()
    df_standard = df.copy()
    df_robust = df.copy()

    # Initialize scalers
    minmax_scaler = MinMaxScaler()
    standard_scaler = StandardScaler()
    robust_scaler = RobustScaler()

    # Apply scaling to numeric columns
    numeric_columns = df.select_dtypes(include=[np.number]).columns

    df_minmax[numeric_columns] = minmax_scaler.fit_transform(df[numeric_columns])
    df_standard[numeric_columns] = standard_scaler.fit_transform(df[numeric_columns])
    df_robust[numeric_columns] = robust_scaler.fit_transform(df[numeric_columns])

    return df_minmax, df_standard, df_robust

# Example usage
data = pd.DataFrame({
    'A': [1, 2, 3, 4, 100],
    'B': [10, 20, 30, 40, 50],
    'C': [-1, -2, 0, 2, 1]
})

minmax_scaled, standard_scaled, robust_scaled = normalize_and_standardize(data)

print("Original Data:\n", data.to_string(index=False))
print("\nMin-Max Scaled:\n", minmax_scaled.to_string(index=False))
print("\nStandardized:\n", standard_scaled.to_string(index=False))
print("\nRobust Scaled: \n", robust_scaled.to_string(index=False))


Original Data:
   A  B  C
  1 10 -1
  2 20 -2
  3 30  0
  4 40  2
100 50  1

Min-Max Scaled:
        A    B    C
0.000000 0.00 0.25
0.010101 0.25 0.00
0.020202 0.50 0.50
0.030303 0.75 1.00
1.000000 1.00 0.75

Standardized:
         A         B         C
-0.538285 -1.414214 -0.707107
-0.512652 -0.707107 -1.414214
-0.487019  0.000000  0.000000
-0.461387  0.707107  1.414214
 1.999343  1.414214  0.707107

Robust Scaled: 
    A    B    C
-1.0 -1.0 -0.5
-0.5 -0.5 -1.0
 0.0  0.0  0.0
 0.5  0.5  1.0
48.5  1.0  0.5


#### Data quality metrics and validation

In [30]:

def assess_data_quality(df):
    quality_report = {}

    # Completeness
    quality_report['completeness'] = df.notna().mean().to_dict()

    # Uniqueness
    quality_report['uniqueness'] = (df.nunique() / len(df)).to_dict()

    # Basic validity checks
    def check_validity(column):
        if pd.api.types.is_numeric_dtype(column):
            return (column >= 0).mean()  # Assuming non-negative values are valid
        elif pd.api.types.is_string_dtype(column):
            return column.str.match(r'^[A-Za-z\s]+$').mean()  # Assuming only letters and spaces are valid
        else:
            return np.nan
        
    quality_report['validity'] = df.apply(check_validity).to_dict()

    # Consistency (example: checking date range)
    if 'date' in df.columns:
        date_range = df['date'].max() - df['date'].min()
        quality_report['date_range_days'] = date_range.days

    return quality_report

# Example usage
data = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'name': ['John Doe', 'Jane Doe', 'Bob Smith', np.nan, '123'],
    'age': [30, 25, 5, 40, 35],
    'date': pd.date_range(start='2023-01-01', periods=5)
})

quality_metrics = assess_data_quality(data)
for metric, values in quality_metrics.items():
    print(f"\n{metric.capitalize()}:")
    if isinstance(values, dict):
        for col, val in values.items():
            print(f"  {col}: {val:.2f}")
    else:
        print(f"  {values}")


Completeness:
  id: 1.00
  name: 0.80
  age: 1.00
  date: 1.00

Uniqueness:
  id: 1.00
  name: 0.80
  age: 1.00
  date: 1.00

Validity:
  id: 1.00
  name: nan
  age: 1.00
  date: nan

Date_range_days:
  4
